In [2]:
import pandas as pd
import os
import numpy as np
import math
from scipy.stats import pearsonr
import requests

pd.set_option('display.max_columns', None)

In [4]:
## Add lineup and Vegas totals to daily player stats

cwd = os.getcwd()
os.chdir(cwd + '/BaselineProjections')

game_stats = pd.read_csv('gamedatawteamobp.csv')

In [20]:
os.chdir('..')
cwd = os.getcwd()
os.chdir(cwd + '/Fantasy.2018-2021')

game_stats18 = pd.read_csv('PlayerGame.2018.csv')
game_stats19 = pd.read_csv('PlayerGame.2019.csv')
game_stats20 = pd.read_csv('PlayerGame.2020.csv')
game_stats = pd.concat([game_stats18, game_stats19, game_stats20], axis=0)
game_stats.head()

/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AtBats,Runs,Hits,Singles,Doubles,Triples,HomeRuns,RunsBattedIn,BattingAverage,Outs,Strikeouts,Walks,HitByPitch,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,StolenBases,CaughtStealing,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage
0,2100777,19,10001213,2,2018,Tim Beckham,BAL,3B,IF,1,NaN,53067,11,TB,2/23/2018 12:00:00 AM,2/23/2018 2:05:00 PM,HOME,1,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.333,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,0.333,0.666,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2100778,19,10000819,2,2018,Ruben Tejada,BAL,SS,IF,0,NaN,53067,11,TB,2/23/2018 12:00:00 AM,2/23/2018 2:05:00 PM,HOME,1,1.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.500,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2100779,19,10006354,2,2018,Trey Mancini,BAL,LF,OF,1,NaN,53067,11,TB,2/23/2018 12:00:00 AM,2/23/2018 2:05:00 PM,HOME,1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333,0.000,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2100780,19,10000129,2,2018,Manny Machado,BAL,SS,IF,1,NaN,53067,11,TB,2/23/2018 12:00:00 AM,2/23/2018 2:05:00 PM,HOME,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.000,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2100781,19,10000497,2,2018,Alex Presley,BAL,CF,OF,0,NaN,53067,11,TB,2/23/2018 12:00:00 AM,2/23/2018 2:05:00 PM,HOME,1,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000,1.000,2.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,6.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
## Parse dataframe to only relevant data

# Select only regular season data
reg_season_game_stats = game_stats.loc[game_stats.SeasonType == 1].reset_index(drop=True)
# Select only batter data
reg_seas_batter_stats = reg_season_game_stats.loc[reg_season_game_stats.PositionCategory != 'P'].reset_index(drop=True)
# Calculate game by game plate appearances
reg_seas_batter_stats['PlateAppearances'] = reg_seas_batter_stats.apply(lambda row: row['AtBats'] + row['Walks'] + row['HitByPitch'] + row['Sacrifices'], axis=1)
# Remove players who appeared in a game, but did not record a PA
reg_seas_batter_stats_nozeros = reg_seas_batter_stats.loc[reg_seas_batter_stats.PlateAppearances > 0]
#reg_seas_batter_stats_nozeros['DraftKingsPointsPerPA'] = reg_seas_batter_stats_nozeros.apply(lambda row: row['FantasyPointsDraftKings'] / row['PlateAppearances'], axis=1)
data = reg_seas_batter_stats_nozeros
data = data.sort_values(['PlayerID', 'Day'], ascending=True).reset_index(drop=True)
data['Day'] = data['Day'].astype('datetime64[ns]')
data.rename(columns = {'Runs': 'R', 'Singles': 'S', 'Doubles': 'D', 'Triples': 'T', 'HomeRuns': 'HR', 'AtBats': 'AB', 'Walks':'BB', 'RunsBattedIn': 'RBI', 'PlateAppearances': 'PA', 'Hits': 'H', 'HitByPitch': 'HP', 'StolenBases': 'SB', 'CaughtStealing': 'CS', 'Strikeouts': 'SO'},  
          inplace = True) 
data.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA
0,2384688,14,10000020,1,2018,Robbie Ray,ARI,PH,PH,0,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2583384,14,10000029,1,2019,Nick Ahmed,ARI,SS,IF,1,NaN,54160,1,LAD,2019-03-28,3/28/2019 4:10:00 PM,AWAY,1,3.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.333,2.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.250,0.667,0.917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5,7.0,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,2157619,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50611,23,COL,2018-03-29,3/29/2018 10:10:00 PM,HOME,1,5.0,3.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.333,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,0.667,1.167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.4,11.0,0.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,2587194,14,10000029,1,2019,Nick Ahmed,ARI,SS,IF,1,NaN,54167,1,LAD,2019-03-29,3/29/2019 10:10:00 PM,AWAY,1,6.0,5.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.200,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.333,0.200,0.533,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.4,14.0,0.333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,2162023,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50618,23,COL,2018-03-30,3/30/2018 9:40:00 PM,HOME,1,12.0,4.0,1.0,3.0,2.0,0.0,0.0,1.0,5.0,0.750,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750,1.500,2.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.7,28.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [6]:
os.chdir('..')
batting_order = pd.read_csv('battingorder2018-2020.csv')
batting_order.rename(columns = {'sports_data_playerid': 'PlayerID', 'sports_data_game_id': 'GameID'}, inplace = True)
batting_order.drop(['full name'], axis = 1, inplace = True)
batting_order.head()

,PlayerID,GameID,battingorderposition
0,10000769,50597,1
1,10000125,50597,1
2,10001184,50598,1
3,10006462,50598,1
4,10001267,50599,1


In [7]:
merged_df = game_stats.merge(batting_order, how = 'inner', on = ['PlayerID', 'GameID'])
merged_df.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA,DraftKingsPointsPerPAActual,InSeasonProjDKPPA,PreSeasonProjDKPPA,Reliability,TotalSeasonPA,BlendedProjDKPPA,pR,pS,pD,pT,pHR,pBB,pHP,pRBI,pSB,playergameID,teamProjectedOBP,battingorderposition
0,2384688,14,10000020,1,2018,Robbie Ray,ARI,PH,PH,0,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000002051985,0.343808,9
1,2162023,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50618,23,COL,2018-03-30,3/30/2018 9:40:00 PM,HOME,1,12.0,4.0,1.0,3.0,2.0,0.0,0.0,1.0,5.0,0.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,1.5,2.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.7,28.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.00,2.750,1.562,0.74,4.0,1.582227,0.473,0.579,0.182,0.024,0.097,0.276,0.024,0.370,0.046,1000002950618,0.348048,7
2,2165293,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50634,23,COL,2018-03-31,3/31/2018 8:10:00 PM,HOME,1,3.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.25,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.25,0.5,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5,7.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.75,4.875,1.562,0.74,8.0,1.672500,0.480,0.600,0.170,0.024,0.120,0.271,0.023,0.436,0.045,1000002950634,0.356789,7
3,2188297,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50759,15,SF,2018-04-10,4/10/2018 10:15:00 PM,AWAY,1,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.50,2.500,1.562,0.74,28.0,1.643539,0.461,0.600,0.180,0.023,0.107,0.282,0.020,0.447,0.040,1000002950759,0.337815,8
4,2190565,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50773,15,SF,2018-04-11,4/11/2018 3:45:00 PM,AWAY,1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.00,2.250,1.562,0.74,32.0,1.625845,0.455,0.592,0.176,0.023,0.104,0.293,0.020,0.441,0.039,1000002950773,0.326048,6


In [8]:
days = merged_df['Day'].tolist()
days = list(dict.fromkeys(days))
days.sort()


In [29]:
game_vegas_lines = []

for day in days:
    t = day.date()
    new_date = t.strftime('%Y-%b-%d')
    
    response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{new_date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
    games = response.json()
    df_games = pd.json_normalize(games, record_path =['DfsSlateGames'])
    
    result_df = df_games.drop_duplicates(subset=['GameID'], keep='first').reset_index(drop=True)
    result_df = result_df[['GameID', 'Game.AwayTeamStartingPitcherID', 'Game.HomeTeamStartingPitcherID', 'Game.PointSpread', 'Game.OverUnder', 'Game.AwayTeamMoneyLine', 'Game.HomeTeamMoneyLine']]
    game_vegas_lines.append(result_df)

print(len(game_vegas_lines))

436


In [13]:
date = new_date

#game slates and salaries

response = requests.get(f'https://api.sportsdata.io/api/mlb/fantasy/json/DfsSlatesByDate/{date}', headers={'Ocp-Apim-Subscription-Key': '6fcab751d8594ce9909283dcdc522d24'})
games = response.json()
df_games = pd.json_normalize(games, record_path =['DfsSlateGames'])
df_games.head()

,SlateGameID,SlateID,GameID,OperatorGameID,RemovedByOperator,Game.GameID,Game.Season,Game.SeasonType,Game.Status,Game.Day,Game.DateTime,Game.AwayTeam,Game.HomeTeam,Game.AwayTeamID,Game.HomeTeamID,Game.StadiumID,Game.AwayTeamRuns,Game.HomeTeamRuns,Game.AwayTeamProbablePitcherID,Game.HomeTeamProbablePitcherID,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine,Game.ForecastTempLow,Game.ForecastTempHigh,Game.ForecastDescription,Game.ForecastWindChill,Game.ForecastWindSpeed,Game.ForecastWindDirection,Game.AwayTeamStartingPitcher,Game.HomeTeamStartingPitcher,Game.HomeRotationNumber,Game.AwayRotationNumber,Game.NeutralVenue,Game.OverPayout,Game.UnderPayout,Game.Innings
0,23967,3582,50606,5515512,None,50606,2018,1,Postponed,2018-03-29T00:00:00,2018-03-29T13:10:00,PIT,DET,4,17,62,None,None,10000843,10001353,10000843.0,NaN,1.5,9.0,-115,105,38,41,Light Rain,35,9,6,None,None,None,None,None,None,None,[]
1,23968,3582,50603,5515838,None,50603,2018,1,Final,2018-03-29T00:00:00,2018-03-29T13:10:00,STL,NYM,31,18,8,None,None,10001157,10000805,10001157.0,10000805.0,-1.5,6.5,123,-133,47,47,Light Rain,43,7,165,None,None,None,None,None,None,None,[]
2,23969,3582,50604,5516366,None,50604,2018,1,Final,2018-03-29T00:00:00,2018-03-29T12:40:00,CHC,MIA,9,22,57,None,None,10000193,10003284,10000193.0,10003284.0,1.5,8.0,-215,195,68,70,Clear Sky,68,8,121,None,None,None,None,None,None,None,[]
3,24071,3596,50604,5516366,None,50604,2018,1,Final,2018-03-29T00:00:00,2018-03-29T12:40:00,CHC,MIA,9,22,57,None,None,10000193,10003284,10000193.0,10003284.0,1.5,8.0,-215,195,68,70,Clear Sky,68,8,121,None,None,None,None,None,None,None,[]
4,24057,3595,50611,5514181,None,50611,2018,1,Final,2018-03-29T00:00:00,2018-03-29T22:10:00,COL,ARI,23,14,54,None,None,10002058,10000007,10002058.0,10000007.0,-1.5,8.0,102,-112,63,64,Clear Sky,64,3,258,None,None,None,None,None,None,None,[]


In [16]:
result_df = df_games.drop_duplicates(subset=['GameID'], keep='first').reset_index(drop=True)
result_df = result_df[['GameID', 'Game.AwayTeamStartingPitcherID', 'Game.HomeTeamStartingPitcherID', 'Game.PointSpread', 'Game.OverUnder', 'Game.AwayTeamMoneyLine', 'Game.HomeTeamMoneyLine']]
result_df.head()

,GameID,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine
0,50606,10000843.0,NaN,1.5,9.0,-115,105
1,50603,10001157.0,10000805.0,-1.5,6.5,123,-133
2,50604,10000193.0,10003284.0,1.5,8.0,-215,195
3,50611,10002058.0,10000007.0,-1.5,8.0,102,-112
4,50600,10005232.0,10000066.0,1.5,8.0,-123,113


In [30]:
all_data = pd.concat(game_vegas_lines)

In [32]:
all_data.shape

(5917, 7)

In [33]:
allall_data = merged_df.merge(all_data, how = 'inner', on = ['GameID'])
allall_data.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA,battingorderposition,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine
0,2384688,14,10000020,1,2018,Robbie Ray,ARI,PH,PH,0,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0
1,2384659,14,10000031,1,2018,Paul Goldschmidt,ARI,1B,IF,1,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,6.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,2.00,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.7,14.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0
2,2384660,14,10000033,1,2018,Jake Lamb,ARI,3B,IF,1,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,3.0,4.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,0.50,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.50,0.50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.2,8.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0
3,2384783,14,10000034,1,2018,Chris Owings,ARI,2B,IF,0,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,9,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0
4,2384662,14,10000040,1,2018,David Peralta,ARI,LF,OF,1,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,2.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.25,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.25,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,5.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0


In [34]:
allall_data.to_csv('all_data.csv')

In [9]:
alldata = pd.read_csv('alldata.csv')
alldata.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA,battingorderposition,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine,PlayerTeamMoneyLine,PlayerTeamPointSpread,PlayerTeamVegasWinProb,PlayerTeamTotal
0,2157290,19,10000121,1,2018,Caleb Joseph,BAL,C,IF,1,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,5.0,3,0,1,0,0,1,0,2,0.333,2,1,0,0,1,0,0,0,0,0.333,1.0,1.333,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,16.0,12,0.667,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,9,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
1,2157281,20,10000481,1,2018,Jason Castro,MIN,C,IF,1,NaN,50597,19,BAL,3/29/18,3/29/18 15:05,AWAY,1,0.0,4,0,0,0,0,0,0,0,0.000,4,2,0,0,0,0,0,0,0,0.000,0.0,0.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,9,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40
2,2157289,19,10000916,1,2018,Craig Gentry,BAL,RF,OF,1,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,0.0,3,0,0,0,0,0,0,0,0.000,3,2,0,0,0,0,0,0,0,0.000,0.0,0.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
3,2157484,19,10001327,1,2018,Colby Rasmus,BAL,RF,OF,0,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,1.0,0,0,0,0,0,0,0,0,0.000,0,0,1,0,0,0,0,0,0,1.000,0.0,1.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,3.0,2,1.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
4,2157465,20,10000494,1,2018,Robbie Grossman,MIN,LF,OF,0,NaN,50597,19,BAL,3/29/18,3/29/18 15:05,AWAY,1,3.0,1,0,1,1,0,0,0,2,1.000,0,0,0,0,0,0,0,0,0,1.000,1.0,2.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,10.0,7,1.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40


In [10]:
alldata['playergameID'] = alldata.apply(lambda row: str(row['PlayerID']) + str(row['GameID']), axis=1)
alldata.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA,battingorderposition,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine,PlayerTeamMoneyLine,PlayerTeamPointSpread,PlayerTeamVegasWinProb,PlayerTeamTotal,playergameID
0,2157290,19,10000121,1,2018,Caleb Joseph,BAL,C,IF,1,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,5.0,3,0,1,0,0,1,0,2,0.333,2,1,0,0,1,0,0,0,0,0.333,1.0,1.333,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,16.0,12,0.667,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,9,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08,1000012150597
1,2157281,20,10000481,1,2018,Jason Castro,MIN,C,IF,1,NaN,50597,19,BAL,3/29/18,3/29/18 15:05,AWAY,1,0.0,4,0,0,0,0,0,0,0,0.000,4,2,0,0,0,0,0,0,0,0.000,0.0,0.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,9,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40,1000048150597
2,2157289,19,10000916,1,2018,Craig Gentry,BAL,RF,OF,1,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,0.0,3,0,0,0,0,0,0,0,0.000,3,2,0,0,0,0,0,0,0,0.000,0.0,0.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08,1000091650597
3,2157484,19,10001327,1,2018,Colby Rasmus,BAL,RF,OF,0,NaN,50597,20,MIN,3/29/18,3/29/18 15:05,HOME,1,1.0,0,0,0,0,0,0,0,0,0.000,0,0,1,0,0,0,0,0,0,1.000,0.0,1.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,3.0,2,1.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08,1000132750597
4,2157465,20,10000494,1,2018,Robbie Grossman,MIN,LF,OF,0,NaN,50597,19,BAL,3/29/18,3/29/18 15:05,AWAY,1,3.0,1,0,1,1,0,0,0,2,1.000,0,0,0,0,0,0,0,0,0,1.000,1.0,2.000,0,0,0,0.0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,10.0,7,1.000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,8,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40,1000049450597


In [11]:
data_to_merge = alldata[['playergameID','Game.AwayTeamStartingPitcherID', 'Game.HomeTeamStartingPitcherID', 'Game.PointSpread', 'Game.OverUnder', 'Game.AwayTeamMoneyLine', 'Game.HomeTeamMoneyLine', 'PlayerTeamMoneyLine', 'PlayerTeamPointSpread', 'PlayerTeamVegasWinProb', 'PlayerTeamTotal']]

data_to_merge.head()


,playergameID,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine,PlayerTeamMoneyLine,PlayerTeamPointSpread,PlayerTeamVegasWinProb,PlayerTeamTotal
0,1000012150597,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
1,1000048150597,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40
2,1000091650597,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
3,1000132750597,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,-125.0,-1.5,0.555556,5.08
4,1000049450597,10001203.0,10000095.0,-1.5,9.5,115.0,-125.0,115.0,1.5,0.465116,4.40


In [21]:
every_data = merged_df.merge(data_to_merge, how = 'inner', on = ['playergameID'])
every_data.head()

,StatID,TeamID,PlayerID,SeasonType,Season,Name,Team,Position,PositionCategory,Started,InjuryStatus,GameID,OpponentID,Opponent,Day,DateTime,HomeOrAway,Games,FantasyPoints,AB,R,H,S,D,T,HR,RBI,BattingAverage,Outs,SO,BB,HP,Sacrifices,SacrificeFlies,GroundIntoDoublePlay,SB,CS,OnBasePercentage,SluggingPercentage,OnBasePlusSlugging,Wins,Losses,Saves,InningsPitchedDecimal,TotalOutsPitched,InningsPitchedFull,InningsPitchedOuts,EarnedRunAverage,PitchingHits,PitchingRuns,PitchingEarnedRuns,PitchingWalks,PitchingStrikeouts,PitchingHomeRuns,PitchesThrown,PitchesThrownStrikes,WalksHitsPerInningsPitched,PitchingBattingAverageAgainst,FantasyPointsFanDuel,FantasyPointsDraftKings,WeightedOnBasePercentage,PitchingCompleteGames,PitchingShutOuts,PitchingOnBasePercentage,PitchingSluggingPercentage,PitchingOnBasePlusSlugging,PitchingStrikeoutsPerNineInnings,PitchingWalksPerNineInnings,PitchingWeightedOnBasePercentage,PA,DraftKingsPointsPerPAActual,InSeasonProjDKPPA,PreSeasonProjDKPPA,Reliability,TotalSeasonPA,BlendedProjDKPPA,pR,pS,pD,pT,pHR,pBB,pHP,pRBI,pSB,playergameID,teamProjectedOBP,battingorderposition,Game.AwayTeamStartingPitcherID,Game.HomeTeamStartingPitcherID,Game.PointSpread,Game.OverUnder,Game.AwayTeamMoneyLine,Game.HomeTeamMoneyLine,PlayerTeamMoneyLine,PlayerTeamPointSpread,PlayerTeamVegasWinProb,PlayerTeamTotal
0,2384688,14,10000020,1,2018,Robbie Ray,ARI,PH,PH,0,NaN,51985,23,COL,2018-07-11,7/11/2018 8:40:00 PM,AWAY,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000002051985,0.343808,9,10000059.0,10005351.0,-1.5,12.0,117.0,-127.0,117.0,1.5,0.460829,5.65
1,2162023,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50618,23,COL,2018-03-30,3/30/2018 9:40:00 PM,HOME,1,12.0,4.0,1.0,3.0,2.0,0.0,0.0,1.0,5.0,0.75,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.75,1.5,2.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.7,28.0,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.00,2.750,1.562,0.74,4.0,1.582227,0.473,0.579,0.182,0.024,0.097,0.276,0.024,0.370,0.046,1000002950618,0.348048,7,10000364.0,10000020.0,-1.5,8.0,130.0,-140.0,-140.0,-1.5,0.583333,4.31
2,2165293,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50634,23,COL,2018-03-31,3/31/2018 8:10:00 PM,HOME,1,3.0,4.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.25,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.25,0.5,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5,7.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.75,4.875,1.562,0.74,8.0,1.672500,0.480,0.600,0.170,0.024,0.120,0.271,0.023,0.436,0.045,1000002950634,0.356789,7,10005351.0,10000613.0,-1.5,8.5,135.0,-145.0,-145.0,-1.5,0.591837,4.56
3,2188297,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50759,15,SF,2018-04-10,4/10/2018 10:15:00 PM,AWAY,1,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,3.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.50,2.500,1.562,0.74,28.0,1.643539,0.461,0.600,0.180,0.023,0.107,0.282,0.020,0.447,0.040,1000002950759,0.337815,8,10000007.0,10005526.0,1.5,7.5,-107.0,-103.0,-107.0,-1.5,0.516908,4.11
4,2190565,14,10000029,1,2018,Nick Ahmed,ARI,SS,IF,1,NaN,50773,15,SF,2018-04-11,4/11/2018 3:45:00 PM,AWAY,1,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.00,2.250,1.562,0.74,32.0,1.625845,0.455,0.592,0.176,0.023,0.104,0.293,0.020,0.441,0.039,1000002950773,0.326048,6,10000020.0,10006816.0,1.5,7.5,-108.0,-102.0,-108.0,-1.5,0.519231,4.11


In [14]:
merged_df.dtypes

StatID                    int64
TeamID                    int64
PlayerID                  int64
SeasonType                int64
Season                    int64
                         ...   
pRBI                    float64
pSB                     float64
playergameID              int64
teamProjectedOBP        float64
battingorderposition      int64
Length: 88, dtype: object

In [19]:
data_to_merge = data_to_merge.astype({'playergameID': 'int64'})

In [20]:
data_to_merge.dtypes

playergameID                        int64
Game.AwayTeamStartingPitcherID    float64
Game.HomeTeamStartingPitcherID    float64
Game.PointSpread                  float64
Game.OverUnder                    float64
Game.AwayTeamMoneyLine            float64
Game.HomeTeamMoneyLine            float64
PlayerTeamMoneyLine               float64
PlayerTeamPointSpread             float64
PlayerTeamVegasWinProb            float64
PlayerTeamTotal                   float64
dtype: object

In [22]:
every_data.to_csv('masterdatafile.csv', index=False)